In [ ]:
from gs_quant.instrument import FXOption
from gs_quant.risk import MarketDataShockBasedScenario, MarketDataPattern, MarketDataShock, MarketDataShockType
from gs_quant.risk import FXSpot, Price, FXAnnualImpliedVol, FXAnnualATMImpliedVol
from gs_quant.session import GsSession

GsSession.use(client_id=None, client_secret=None)

In [ ]:
fxoption = FXOption(pair='EURUSD', expiration_date='3m', premium=0)
unshifted_res = fxoption.calc((FXSpot, Price, FXAnnualImpliedVol, FXAnnualATMImpliedVol))
unshifted_res.to_frame()

In [ ]:
# set FX Spot to X

x = 1.1

mkt_shock = MarketDataShockBasedScenario(shocks={MarketDataPattern('FX', 'USD/EUR'): 
                                                 MarketDataShock(MarketDataShockType.Override, x)})

with mkt_shock:
    shifted_spot = fxoption.calc(FXSpot)
shifted_spot

In [ ]:
# Bump FX spot by x

# relative - 5%
x = 0.05

mkt_shock = MarketDataShockBasedScenario(shocks={MarketDataPattern('FX', 'USD/EUR'): 
                                                 MarketDataShock(MarketDataShockType.Proportional, x)})

with mkt_shock:
    shifted_spot = fxoption.calc(FXSpot)
print(shifted_spot)
print(unshifted_res[FXSpot] * (1 + x))

# absolute - 0.08
x = 0.08

mkt_shock = MarketDataShockBasedScenario(shocks={MarketDataPattern('FX', 'USD/EUR'): 
                                                 MarketDataShock(MarketDataShockType.Absolute, x)})

with mkt_shock:
    shifted_spot = fxoption.calc(FXSpot)
print(shifted_spot)
print(unshifted_res[FXSpot] + x)

In [ ]:
# Set Strike Vol to x  & Bump Strike Vol to x

x = 0.06

mkt_shock = MarketDataShockBasedScenario(shocks={MarketDataPattern('FX VOL', 'USD/EUR', 'ATM VOL'): 
                                                 MarketDataShock(MarketDataShockType.Override, x),
                                                 MarketDataPattern('FX VOL', 'USD/EUR', 'BF 25'): 
                                                 MarketDataShock(MarketDataShockType.Override, 0),
                                                 MarketDataPattern('FX VOL', 'USD/EUR', 'RR 25'): 
                                                 MarketDataShock(MarketDataShockType.Override, 0),})

with mkt_shock:
    shifted_vol = fxoption.calc(FXAnnualImpliedVol)
print(shifted_vol)

In [ ]:
# Parallel Shift the ATMVol Curve by x

# relative - 5%
x = 0.05

mkt_shock = MarketDataShockBasedScenario(shocks={MarketDataPattern('FX VOL', 'USD/EUR', 'ATM VOL'): 
                                                 MarketDataShock(MarketDataShockType.Proportional, x)})

with mkt_shock:
    shifted_vol = fxoption.calc(FXAnnualATMImpliedVol)
print(shifted_vol)
print(unshifted_res[FXAnnualATMImpliedVol] * (1 + x))

# absolute - 0.01
x = 0.01

mkt_shock = MarketDataShockBasedScenario(shocks={MarketDataPattern('FX VOL', 'USD/EUR', 'ATM VOL'): 
                                                 MarketDataShock(MarketDataShockType.Absolute, x)})

with mkt_shock:
    shifted_vol = fxoption.calc(FXAnnualATMImpliedVol)
print(shifted_vol)
print(unshifted_res[FXAnnualATMImpliedVol] + x)

In [ ]:
# set atmvol at expiry X to y

x = '3m'
y = 0.62

mkt_shock = MarketDataShockBasedScenario(shocks={MarketDataPattern('FX VOL', 'USD/EUR', 'ATM VOL', mkt_point=[x]): 
                                                 MarketDataShock(MarketDataShockType.Override, y)})

with mkt_shock:
    shifted_atmvol = fxoption.calc(FXAnnualATMImpliedVol)
shifted_atmvol

In [ ]:
# Parallel Shift the ATMVol at expiry x by y

x = '3m'
# relative - 5%
y = 0.05

mkt_shock = MarketDataShockBasedScenario(shocks={MarketDataPattern('FX VOL', 'USD/EUR', 'ATM VOL', mkt_point=[x]): 
                                                 MarketDataShock(MarketDataShockType.Proportional, y)})

with mkt_shock:
    shifted_vol = fxoption.calc(FXAnnualATMImpliedVol)
print(shifted_vol)
print(unshifted_res[FXAnnualATMImpliedVol] * (1 + y))

# absolute - 0.01
y = 0.01

mkt_shock = MarketDataShockBasedScenario(shocks={MarketDataPattern('FX VOL', 'USD/EUR', 'ATM VOL', mkt_point=[x]): 
                                                 MarketDataShock(MarketDataShockType.Absolute, y)})

with mkt_shock:
    shifted_vol = fxoption.calc(FXAnnualATMImpliedVol)
print(shifted_vol)
print(unshifted_res[FXAnnualATMImpliedVol] + y)

In [ ]:
# parallel shift the yield curve

ir_spot_scenario = MarketDataShockBasedScenario(shocks={MarketDataPattern('IR', 'EUR'): 
                                                        MarketDataShock(MarketDataShockType.Absolute, 50 / 10000)})

with ir_spot_scenario:
    yield_curve_shift = fxoption.price()

print(yield_curve_shift)
print(unshifted_res[Price])
